[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/brandonmccraryresearch-cloud/Intrinsic_Resonace_Holography-/blob/main/notebooks/04_falsification_analysis.ipynb) [![GitHub](https://img.shields.io/badge/GitHub-Repository-blue?logo=github)](https://github.com/brandonmccraryresearch-cloud/Intrinsic_Resonace_Holography-)

# IRH v21.1: Falsification Analysis

**THEORETICAL FOUNDATION**: [Intrinsic_Resonance_Holography-v21.1.md](https://github.com/brandonmccraryresearch-cloud/Intrinsic_Resonace_Holography-/blob/main/Intrinsic_Resonance_Holography-v21.1.md) §8, Appendix J

This notebook analyzes the falsifiable predictions of IRH v21.1 and
compares them with current and upcoming experimental constraints.

---

## Contents

1. [Setup](#1-setup)
2. [Dark Energy Equation of State](#2-dark-energy-equation-of-state)
3. [Lorentz Invariance Violation](#3-lorentz-invariance-violation)
4. [Neutrino Mass Hierarchy](#4-neutrino-mass-hierarchy)
5. [Muon g-2 Anomaly](#5-muon-g-2-anomaly)
6. [Falsification Timeline](#6-falsification-timeline)

## 1. Setup

In [ ]:
# Install IRH on Colab
import sys
if 'google.colab' in sys.modules:
    print("Running on Google Colab - Installing IRH...")
    !git clone https://github.com/brandonmccraryresearch-cloud/Intrinsic_Resonace_Holography-.git /content/IRH
    sys.path.insert(0, '/content/IRH')
    %cd /content/IRH
    !pip install -q numpy scipy sympy matplotlib
    print("\n✓ IRH installed successfully!")
else:
    print("Running locally")

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Import IRH modules
from src.cosmology.dark_energy import compute_dark_energy_eos
from src.falsifiable_predictions.lorentz_violation import compute_liv_parameter
from src.standard_model.neutrinos import compute_neutrino_masses, neutrino_hierarchy
from src.falsifiable_predictions.muon_g_minus_2 import compute_muon_g_minus_2

print("✓ Modules imported successfully!")

## 2. Dark Energy Equation of State

IRH predicts a specific dark energy equation of state:

$$w_0 = -0.91234567 \pm 0.00000008$$

This deviates significantly from ΛCDM ($w = -1$) and can be tested by:
- **Euclid** (2024-2029)
- **Nancy Grace Roman** (2026+)
- **DESI** (2021-2026)

**Reference**: Intrinsic_Resonance_Holography-v21.1.md §2.3.3, Eqs. 2.21-2.23

In [ ]:
# Compute dark energy equation of state
de_result = compute_dark_energy_eos()

print("Dark Energy Equation of State (§2.3)")
print("="*60)
print(f"IRH Prediction: w₀ = {de_result.w0:.8f} ± {de_result.uncertainty:.8f}")
print(f"ΛCDM:           w₀ = -1.000")
print(f"Current data:   w₀ = -1.03 ± 0.03 (Planck + BAO + SNe)")
print(f"\nDeviation from ΛCDM: {abs(de_result.w0 - (-1.0)):.3f}")
print(f"Phantom threshold:   w < -1 (IRH predicts NON-phantom!)")

In [ ]:
# Visualize w₀ constraints
fig, ax = plt.subplots(figsize=(10, 6))

# Current constraints
experiments = {
    'Planck 2018': (-1.03, 0.03),
    'DES Y3': (-0.95, 0.08),
    'DESI 2024': (-0.99, 0.05),
    'IRH Prediction': (de_result.w0, de_result.uncertainty),
    'ΛCDM': (-1.0, 0.0),
}

y_positions = np.arange(len(experiments))
colors = ['gray', 'gray', 'gray', 'blue', 'red']

for i, (name, (w0, err)) in enumerate(experiments.items()):
    ax.errorbar(w0, i, xerr=err, fmt='o', capsize=5, markersize=10, 
                color=colors[i], label=name)
    ax.annotate(f'{w0:.3f} ± {err:.3f}' if err > 0 else f'{w0:.3f}', 
                (w0 + 0.05, i + 0.1), fontsize=9)

ax.axvline(-1.0, color='red', linestyle='--', alpha=0.5, label='ΛCDM (w=-1)')
ax.axvline(de_result.w0, color='blue', linestyle='--', alpha=0.5)
ax.fill_betweenx([-1, 5], -1.1, -1.0, alpha=0.1, color='gray', label='Phantom regime')

ax.set_yticks(y_positions)
ax.set_yticklabels(experiments.keys())
ax.set_xlabel('w₀ (equation of state parameter)')
ax.set_title('Dark Energy Equation of State: IRH vs Observations')
ax.set_xlim(-1.2, -0.7)
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"\nFalsification: If w₀ = -1.0 ± 0.01 is confirmed, IRH is falsified.")
print(f"Test timeline: Euclid/Roman will reach precision σ(w₀) ~ 0.01 by 2029.")

## 3. Lorentz Invariance Violation

IRH predicts a specific LIV parameter:

$$\xi = \frac{C_H}{24\pi^2} \approx 1.93 \times 10^{-4}$$

This leads to energy-dependent photon time delays testable via:
- **CTA** (Cherenkov Telescope Array)
- **Fermi-LAT** high-energy gamma rays

**Reference**: Intrinsic_Resonance_Holography-v21.1.md §2.5, Eqs. 2.24-2.26

In [ ]:
# Compute LIV parameter
liv_result = compute_liv_parameter()

print("Lorentz Invariance Violation (§2.5)")
print("="*60)
print(f"IRH Prediction: ξ = {liv_result.xi:.2e}")
print(f"Formula: ξ = C_H/(24π²)")
print(f"\nModified dispersion relation:")
print(f"  E² = p²c² + ξ × E³/(E_Planck × c²)")
print(f"\nPhoton time delay (for GRB at z=1, E=TeV):")
print(f"  Δt ≈ ξ × (E/E_Planck) × d/c ≈ {liv_result.xi * 1e-16 * 1e28:.1f} seconds")

In [ ]:
# LIV parameter comparison
fig, ax = plt.subplots(figsize=(10, 5))

# Constraints and predictions
bounds = {
    'Fermi-LAT (GRB090510)': (1e-1, 'upper'),
    'MAGIC (Mrk501)': (1e-2, 'upper'),
    'CTA sensitivity': (1e-3, 'projected'),
    'IRH Prediction': (liv_result.xi, 'prediction'),
}

positions = np.arange(len(bounds))
colors = {'upper': 'gray', 'projected': 'orange', 'prediction': 'blue'}

for i, (name, (val, btype)) in enumerate(bounds.items()):
    ax.barh(i, np.log10(val), color=colors[btype], alpha=0.7)
    ax.annotate(f'{val:.2e}', (np.log10(val) + 0.1, i), fontsize=10, va='center')

ax.set_yticks(positions)
ax.set_yticklabels(bounds.keys())
ax.set_xlabel('log₁₀(ξ)')
ax.set_title('LIV Parameter ξ: Constraints and IRH Prediction')
ax.axvline(np.log10(liv_result.xi), color='blue', linestyle='--', alpha=0.7)
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"\nFalsification: If ξ < 10⁻⁵ is established, IRH is falsified.")
print(f"Test timeline: CTA will reach required sensitivity by 2028-2029.")

## 4. Neutrino Mass Hierarchy

IRH predicts:
- **Normal hierarchy** (m₁ < m₂ < m₃)
- **Majorana nature** (from topology)
- **Sum of masses**: Σm_ν ≈ 0.058 eV

**Reference**: Intrinsic_Resonance_Holography-v21.1.md §3.2.4, Appendix E.3

In [ ]:
# Compute neutrino predictions
hierarchy = neutrino_hierarchy()
masses = compute_neutrino_masses()

print("Neutrino Sector (§3.2.4)")
print("="*60)
print(f"Mass Hierarchy: {hierarchy.upper()}")
print(f"\nIndividual masses:")
print(f"  m₁ = {masses.m1_eV:.6f} eV")
print(f"  m₂ = {masses.m2_eV:.6f} eV")
print(f"  m₃ = {masses.m3_eV:.6f} eV")
print(f"\nSum: Σm_ν = {masses.sum_masses:.4f} eV")
print(f"\nCosmological bound: Σm_ν < 0.12 eV (Planck 2018)")
print(f"IRH satisfies bound: {masses.sum_masses < 0.12}")

In [ ]:
# Neutrino mass hierarchy visualization
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))

# Mass spectrum
masses_list = [masses.m1_eV, masses.m2_eV, masses.m3_eV]
ax1.bar(['m₁', 'm₂', 'm₃'], masses_list, color=['blue', 'green', 'red'])
ax1.set_ylabel('Mass (eV)')
ax1.set_title('Neutrino Mass Spectrum (Normal Hierarchy)')
ax1.axhline(0.12/3, color='gray', linestyle='--', alpha=0.5, label='Cosmological bound / 3')
ax1.legend()

# Hierarchy comparison
ax2.bar(['Normal', 'Inverted'], [1, 0], color=['green', 'gray'], alpha=0.7)
ax2.set_ylabel('IRH Prediction')
ax2.set_title('Mass Hierarchy Prediction')
ax2.set_ylim(0, 1.2)

plt.tight_layout()
plt.show()

print(f"\nFalsification: If INVERTED hierarchy is confirmed, IRH is falsified.")
print(f"Test timeline: JUNO will determine hierarchy by 2028-2029.")

## 5. Muon g-2 Anomaly

IRH predicts a specific contribution to the muon anomalous magnetic moment.

**Reference**: Intrinsic_Resonance_Holography-v21.1.md Appendix J.3

In [ ]:
# Compute muon g-2
g2_result = compute_muon_g_minus_2()

print("Muon g-2 (Appendix J.3)")
print("="*60)
print(f"IRH contribution: a_μ(IRH) = {g2_result.a_mu_irh:.2e}")
print(f"\nExperimental values:")
print(f"  a_μ(exp) = 116592061(41) × 10⁻¹¹ (FNAL 2023)")
print(f"  a_μ(SM) = 116591810(43) × 10⁻¹¹ (theory avg.)")
print(f"  Δa_μ = (251 ± 59) × 10⁻¹¹ (anomaly)")

## 6. Falsification Timeline

Summary of IRH predictions and their experimental tests:

In [ ]:
# Falsification timeline
tests = [
    ('Dark energy w₀', 'w₀ = -0.912', 'Euclid/Roman', 2029, 'Critical'),
    ('LIV parameter ξ', 'ξ = 1.93×10⁻⁴', 'CTA', 2029, 'Critical'),
    ('Neutrino hierarchy', 'NORMAL', 'JUNO', 2028, 'Critical'),
    ('Muon g-2', 'VWP contribution', 'Fermilab', 2025, 'Supporting'),
    ('Higgs trilinear', '~5% deviation', 'HL-LHC', 2030, 'Supporting'),
    ('GW sidebands', 'f_sideband', 'LIGO/ET', 2032, 'Supporting'),
]

print("IRH Falsification Timeline (2025-2032)")
print("="*80)
print(f"{'Prediction':<25} {'IRH Value':<20} {'Experiment':<15} {'Year':<6} {'Priority':<10}")
print("-"*80)
for pred, val, exp, year, priority in tests:
    print(f"{pred:<25} {val:<20} {exp:<15} {year:<6} {priority:<10}")

In [ ]:
# Timeline visualization
fig, ax = plt.subplots(figsize=(14, 6))

years = [2025, 2026, 2027, 2028, 2029, 2030, 2031, 2032]
ax.set_xlim(2024.5, 2032.5)
ax.set_ylim(0, 7)

# Plot experiments
experiments_timeline = [
    ('Muon g-2 (FNAL)', 2025, 6, 'green'),
    ('JUNO (neutrinos)', 2028, 5, 'blue'),
    ('CTA (LIV)', 2029, 4, 'red'),
    ('Euclid (w₀)', 2029, 3, 'purple'),
    ('HL-LHC (Higgs)', 2030, 2, 'orange'),
    ('Einstein Telescope', 2032, 1, 'gray'),
]

for name, year, y, color in experiments_timeline:
    ax.scatter(year, y, s=200, c=color, zorder=5)
    ax.annotate(name, (year + 0.1, y + 0.1), fontsize=10)
    ax.hlines(y, 2025, year, colors=color, alpha=0.3, linestyles='--')

ax.axvline(2025, color='green', alpha=0.2, linewidth=20)
ax.axvline(2029, color='red', alpha=0.2, linewidth=20)

ax.set_xlabel('Year', fontsize=12)
ax.set_yticks([])
ax.set_title('IRH Falsification Timeline: Key Experiments', fontsize=14)

# Add legend
ax.text(2024.7, 6.5, '2025: First tests', fontsize=10, color='green')
ax.text(2024.7, 0.5, '2029: Critical tests', fontsize=10, color='red')

plt.tight_layout()
plt.show()

---

## Summary

IRH makes specific, falsifiable predictions:

| Prediction | IRH Value | Test | Year |
|------------|-----------|------|------|
| w₀ (dark energy) | -0.912 | Euclid/Roman | 2029 |
| ξ (LIV) | 1.93×10⁻⁴ | CTA | 2029 |
| Neutrino hierarchy | NORMAL | JUNO | 2028 |
| Higgs trilinear | ~5% from SM | HL-LHC | 2030 |

### Falsification Criteria

IRH would be **falsified** if:
1. w₀ = -1.00 ± 0.01 is confirmed (exact ΛCDM)
2. ξ < 10⁻⁵ is established (no LIV)
3. Inverted neutrino hierarchy is confirmed

### Citation

```bibtex
@software{IRH_v21_computational_2025,
  title={Intrinsic Resonance Holography v21.1: Computational Framework},
  author={McCrary, Brandon D.},
  year={2025},
  url={https://github.com/brandonmccraryresearch-cloud/Intrinsic_Resonace_Holography-}
}
```